# Digital PDF Signing with p12 Certificate
First, let's install required packages

In [1]:
!pip install pypdf reportlab endesive

  Installing build dependencies ... -

 \

 |

 /

 done


...

 \

 |

 /

 -

 \ done
  Created wheel for pykcs11: filename=pykcs11-1.5.17-cp313-cp313-linux_x86_64.whl size=1410444 sha256=9bebef7610bd394a669b3fa1d6f1523c8d4b65ccece1efd1afd7a01ccad2f007
  Stored in directory: /home/runner/.cache/pip/wheels/23/0b/d6/39724a8f324681d01b6ba996221ac7fe35bc39f64ca5d568f2
Successfully built pykcs11


Import necessary libraries and set up timestamp

In [2]:
from pypdf import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from endesive import pdf
import io
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
signature_text = f"Digitally signed by R. Haase, {timestamp}"

LibraryNotFoundError: Error detecting the version of libcrypto

Create visible signature watermark

In [ ]:
packet = io.BytesIO()
c = canvas.Canvas(packet, pagesize=letter)
c.setFont("Helvetica", 12)
c.setFillColorRGB(1, 0, 0)
c.drawString(50, 50, signature_text)
c.save()
packet.seek(0)

Add visible signature to PDF

In [ ]:
existing_pdf = PdfReader("test2.pdf")
signature_pdf = PdfReader(packet)
output = PdfWriter()

page = existing_pdf.pages[0]
page.merge_page(signature_pdf.pages[0])
output.add_page(page)

for page in existing_pdf.pages[1:]:
    output.add_page(page)

output.write("temp_signed.pdf")

Add cryptographic signature using p12 certificate

In [ ]:
date = datetime.utcnow()
date_str = date.strftime("%Y%m%d%H%M%S+00'00'")

with open("certificate.p12", "rb") as fp:
    p12 = fp.read()

dct = {
    "sigflags": 3,
    "contact": "R. Haase",
    "location": "Dresden",
    "signingdate": date_str,
    "reason": "Digital Signature",
}

with open("temp_signed.pdf", "rb") as fp:
    datau = fp.read()

signature = pdf.cms.sign(datau, dct,
    p12,
    "certificate_password",  # Replace with actual password
    "sha256"
)

with open("signed_test2.pdf", "wb") as fp:
    fp.write(datau)
    fp.write(signature)